#Feature Engineering

If you are new to Apache Spark , I highly suggest checking out my kaggle kernel https://www.kaggle.com/lpdataninja/machine-learning-with-apache-spark to get started.

#### What is Feature Engineering ?

Feature engineering is the process of transforming raw data into features that better represent the underlying problem to the predictive models, resulting in improved model accuracy on unseen data.

Feature Engineering is a key part in any ML problem and it will play a major role getting best machine learning model.

#### Let's start with importing the necessary modules of pyspark.

In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

#### Importing data 


Download dataset from Kaggle https://www.kaggle.com/c/home-credit-default-risk/data

#### Interpreting Bureau Data

Information about client's previous loans with other financial institutions reported to Home Credit. Each previous loan has its own row.

In [8]:
bureau = spark.read.csv("/mnt/lp-dataset/HomeCreditDefaultRisk/bureau.csv",header = 'True',inferSchema='True')

In the example below customer with SK_ID_CURR = 100001 had 7 credit transactions before the current application.

In [10]:
display(bureau.where("SK_ID_CURR = 100001"))

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
100001,5896630,Closed,currency 1,-857,0,-492.0,-553.0,null,0,112500.0,0.0,0.0,0.0,Consumer credit,-155,0.0
100001,5896631,Closed,currency 1,-909,0,-179.0,-877.0,null,0,279720.0,0.0,0.0,0.0,Consumer credit,-155,0.0
100001,5896632,Closed,currency 1,-879,0,-514.0,-544.0,null,0,91620.0,0.0,0.0,0.0,Consumer credit,-155,0.0
100001,5896633,Closed,currency 1,-1572,0,-1329.0,-1328.0,null,0,85500.0,0.0,0.0,0.0,Consumer credit,-155,0.0
100001,5896634,Active,currency 1,-559,0,902.0,null,null,0,337680.0,113166.0,0.0,0.0,Consumer credit,-6,4630.5
100001,5896635,Active,currency 1,-49,0,1778.0,null,null,0,378000.0,373239.0,0.0,0.0,Consumer credit,-16,10822.5
100001,5896636,Active,currency 1,-320,0,411.0,null,null,0,168345.0,110281.5,null,0.0,Consumer credit,-10,9364.5


Let's see the Schema of th Bureau Data

In [12]:
bureau.printSchema()

root
-- SK_ID_CURR: integer (nullable = true)
-- SK_ID_BUREAU: integer (nullable = true)
-- CREDIT_ACTIVE: string (nullable = true)
-- CREDIT_CURRENCY: string (nullable = true)
-- DAYS_CREDIT: integer (nullable = true)
-- CREDIT_DAY_OVERDUE: integer (nullable = true)
-- DAYS_CREDIT_ENDDATE: double (nullable = true)
-- DAYS_ENDDATE_FACT: double (nullable = true)
-- AMT_CREDIT_MAX_OVERDUE: double (nullable = true)
-- CNT_CREDIT_PROLONG: integer (nullable = true)
-- AMT_CREDIT_SUM: double (nullable = true)
-- AMT_CREDIT_SUM_DEBT: double (nullable = true)
-- AMT_CREDIT_SUM_LIMIT: double (nullable = true)
-- AMT_CREDIT_SUM_OVERDUE: double (nullable = true)
-- CREDIT_TYPE: string (nullable = true)
-- DAYS_CREDIT_UPDATE: integer (nullable = true)
-- AMT_ANNUITY: double (nullable = true)

#### Understanstanding of variables

CREDIT_ACTIVE - Current status of a Loan - Closed/ Active (2 values)

CREDIT_CURRENCY - Currency in which the transaction was executed - Currency1, Currency2, Currency3, Currency4 ( 4 values)

CREDIT_DAY_OVERDUE - Number of overdue days

CREDIT_TYPE - Consumer Credit, Credit card, Mortgage, Car loan, Microloan, Loan for working capital replemishment, Loan for Business development, Real estate loan, Unkown type of laon, Another type of loan. Cash loan, Loan for the purchase of equipment, Mobile operator loan, Interbank credit, Loan for purchase of shares ( 15 values )

DAYS_CREDIT - Number of days ELAPSED since customer applied for CB credit with respect to current application Interpretation - Are these loans evenly spaced time intervals? Are they concentrated within a same time frame?

DAYS_CREDIT_ENDDATE - Number of days the customer CREDIT is valid at the time of application CREDIT_DAY_OVERDUE - Number of days the customer CREDIT is past the end date at the time of application

AMT_CREDIT_SUM - Total available credit for a customer AMT_CREDIT_SUM_DEBT - Total amount yet to be repayed AMT_CREDIT_SUM_LIMIT - Current Credit that has been utilized AMT_CREDIT_SUM_OVERDUE - Current credit payment that is overdue CNT_CREDIT_PROLONG - How many times was the Credit date prolonged

Total number of transactions in bureau dat

In [15]:
print(bureau.count())

1716428

#### Feature 1 : Numer of past loans per custermer

In [17]:
bureau_10000 = bureau.limit(10000) #limit number number of rows to 10000
loans_per_customer = bureau_10000.select('SK_ID_CURR', 'DAYS_CREDIT').groupBy('SK_ID_CURR').count().withColumnRenamed("count", "BUREAU_LOAN_COUNT")
bureau_10000 = bureau_10000.join(loans_per_customer, ['SK_ID_CURR'],how ='left')
print((bureau_10000.count(), len(bureau_10000.columns)))

(10000, 18)

In [18]:
display(bureau_10000)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_LOAN_COUNT
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,7
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,7
215354,5714464,Active,currency 1,-203,0,528.0,null,null,0,464323.5,null,null,0.0,Consumer credit,-16,null,7
215354,5714465,Active,currency 1,-203,0,null,null,null,0,90000.0,null,null,0.0,Credit card,-16,null,7
215354,5714466,Active,currency 1,-629,0,1197.0,null,77674.5,0,2700000.0,null,null,0.0,Consumer credit,-21,null,7
215354,5714467,Active,currency 1,-273,0,27460.0,null,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,null,7
215354,5714468,Active,currency 1,-43,0,79.0,null,0.0,0,42103.8,42103.8,0.0,0.0,Consumer credit,-22,null,7
162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.0,0.0,0.0,Consumer credit,-1710,null,6
162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.7,0.0,0.0,0.0,Consumer credit,-840,null,6
162297,5714471,Active,currency 1,-1146,0,-484.0,null,0.0,0,4500.0,0.0,0.0,0.0,Credit card,-690,null,6


#### Feature 2 : Numer of types of past loans per custermer

In [20]:
loans_type_per_customer = bureau_10000.select('SK_ID_CURR', 'CREDIT_TYPE').groupBy('SK_ID_CURR').agg(countDistinct("CREDIT_TYPE")).withColumnRenamed("count(DISTINCT CREDIT_TYPE)", "BUREAU_LOAN_TYPES")
bureau_10000 = bureau_10000.join(loans_type_per_customer, ['SK_ID_CURR'], how ='left')

In [21]:
display(bureau_10000)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_LOAN_COUNT,BUREAU_LOAN_TYPES
100053,5723577,Closed,currency 1,-1764,0,-648.0,-648.0,null,0,135000.0,0.0,0.0,0.0,Consumer credit,-644,null,7,2
100053,5723578,Active,currency 1,-2338,0,-1958.0,null,null,0,48820.5,null,null,0.0,Consumer credit,-1945,null,7,2
100053,5723579,Closed,currency 1,-2688,0,-2534.0,-2551.0,null,0,26919.0,0.0,null,0.0,Consumer credit,-1723,null,7,2
100053,5723580,Closed,currency 1,-2893,0,null,-1588.0,null,0,0.0,0.0,null,0.0,Credit card,-1588,null,7,2
100053,5723581,Closed,currency 1,-2639,0,-2496.0,-2496.0,null,0,77850.0,0.0,null,0.0,Credit card,-729,null,7,2
100053,5723582,Closed,currency 1,-1925,0,-832.0,-707.0,null,0,225000.0,null,null,0.0,Consumer credit,-704,null,7,2
100053,5723583,Closed,currency 1,-1767,0,-648.0,-630.0,null,0,135000.0,null,null,0.0,Consumer credit,-630,null,7,2
100568,5719892,Closed,currency 1,-556,0,-130.0,-222.0,null,0,64363.5,0.0,null,0.0,Consumer credit,-201,null,7,2
100568,5719893,Closed,currency 1,-207,0,-26.0,-26.0,null,0,37480.5,0.0,0.0,0.0,Consumer credit,-25,0.0,7,2
100568,5719894,Closed,currency 1,-1274,0,-969.0,-965.0,null,0,75375.0,0.0,0.0,0.0,Consumer credit,-964,0.0,7,2


#### Feature 3 : Average number of past loans per type of customer
Is the Customer diversified in taking multiple types of Loan or Focused on a single type of loan ?

In [23]:
bureau_10000 = bureau_10000.withColumn("AVERAGE_LOAN_TYPE",col("BUREAU_LOAN_COUNT")/col("BUREAU_LOAN_TYPES"))

In [24]:
display(bureau_10000)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_LOAN_COUNT,BUREAU_LOAN_TYPES,AVERAGE_LOAN_TYPE
100053,5723577,Closed,currency 1,-1764,0,-648.0,-648.0,null,0,135000.0,0.0,0.0,0.0,Consumer credit,-644,null,7,2,3.5
100053,5723578,Active,currency 1,-2338,0,-1958.0,null,null,0,48820.5,null,null,0.0,Consumer credit,-1945,null,7,2,3.5
100053,5723579,Closed,currency 1,-2688,0,-2534.0,-2551.0,null,0,26919.0,0.0,null,0.0,Consumer credit,-1723,null,7,2,3.5
100053,5723580,Closed,currency 1,-2893,0,null,-1588.0,null,0,0.0,0.0,null,0.0,Credit card,-1588,null,7,2,3.5
100053,5723581,Closed,currency 1,-2639,0,-2496.0,-2496.0,null,0,77850.0,0.0,null,0.0,Credit card,-729,null,7,2,3.5
100053,5723582,Closed,currency 1,-1925,0,-832.0,-707.0,null,0,225000.0,null,null,0.0,Consumer credit,-704,null,7,2,3.5
100053,5723583,Closed,currency 1,-1767,0,-648.0,-630.0,null,0,135000.0,null,null,0.0,Consumer credit,-630,null,7,2,3.5
100568,5719892,Closed,currency 1,-556,0,-130.0,-222.0,null,0,64363.5,0.0,null,0.0,Consumer credit,-201,null,7,2,3.5
100568,5719893,Closed,currency 1,-207,0,-26.0,-26.0,null,0,37480.5,0.0,0.0,0.0,Consumer credit,-25,0.0,7,2,3.5
100568,5719894,Closed,currency 1,-1274,0,-969.0,-965.0,null,0,75375.0,0.0,0.0,0.0,Consumer credit,-964,0.0,7,2,3.5


#### Feature 4 : % of active loans

In [26]:
def credit_active_close(x):
    if x == 'Closed':
        y = 0
    else:
        y = 1    
    return y

In [27]:
credit_active_close_udf = udf( credit_active_close, StringType() )

In [28]:
bureau_10000 = bureau.limit(10000) #limit number number of rows to 10000
bureau_10000 = bureau_10000.withColumn( "CREDIT_ACTIVE_BINARY", credit_active_close_udf( bureau_10000.CREDIT_ACTIVE) )
# Calculate mean number of loans that are ACTIVE per CUSTOMER 
credit_active_avg = bureau_10000.groupBy("SK_ID_CURR").agg(avg("CREDIT_ACTIVE_BINARY").alias("ACTIVE_LOANS_PERCENTAGE"))
bureau_10000 = bureau_10000.join(credit_active_avg, ['SK_ID_CURR'], how ='left')

In [29]:
display(bureau_10000)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,CREDIT_ACTIVE_BINARY,ACTIVE_LOANS_PERCENTAGE
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,0,0.8571428571428571
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,1,0.8571428571428571
215354,5714464,Active,currency 1,-203,0,528.0,null,null,0,464323.5,null,null,0.0,Consumer credit,-16,null,1,0.8571428571428571
215354,5714465,Active,currency 1,-203,0,null,null,null,0,90000.0,null,null,0.0,Credit card,-16,null,1,0.8571428571428571
215354,5714466,Active,currency 1,-629,0,1197.0,null,77674.5,0,2700000.0,null,null,0.0,Consumer credit,-21,null,1,0.8571428571428571
215354,5714467,Active,currency 1,-273,0,27460.0,null,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,null,1,0.8571428571428571
215354,5714468,Active,currency 1,-43,0,79.0,null,0.0,0,42103.8,42103.8,0.0,0.0,Consumer credit,-22,null,1,0.8571428571428571
162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.0,0.0,0.0,Consumer credit,-1710,null,0,0.5
162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.7,0.0,0.0,0.0,Consumer credit,-840,null,0,0.5
162297,5714471,Active,currency 1,-1146,0,-484.0,null,0.0,0,4500.0,0.0,0.0,0.0,Credit card,-690,null,1,0.5


#### Feature 5 : Average number of days between successive past applications for each customer

How often did the customer take credit in the past? Was it spaced out at regular time intervals - a signal of good financial planning OR were the loans concentrated around a smaller time frame - indicating potential financial trouble ?

In [31]:
bureau_10000 = bureau.limit(10000)
customer_credit = bureau_10000.select("SK_ID_CURR", "SK_ID_BUREAU", "DAYS_CREDIT")
customer_credit = customer_credit.withColumn("DAYS_CREDIT_P",col("DAYS_CREDIT")* lit(-1))

In [32]:
customer_credit.show()

+----------+------------+-----------+-------------+
SK_ID_CURR|SK_ID_BUREAU|DAYS_CREDIT|DAYS_CREDIT_P|
+----------+------------+-----------+-------------+
 215354| 5714462| -497| 497|
 215354| 5714463| -208| 208|
 215354| 5714464| -203| 203|
 215354| 5714465| -203| 203|
 215354| 5714466| -629| 629|
 215354| 5714467| -273| 273|
 215354| 5714468| -43| 43|
 162297| 5714469| -1896| 1896|
 162297| 5714470| -1146| 1146|
 162297| 5714471| -1146| 1146|
 162297| 5714472| -1146| 1146|
 162297| 5714473| -2456| 2456|
 162297| 5714474| -277| 277|
 402440| 5714475| -96| 96|
 238881| 5714482| -318| 318|
 238881| 5714484| -2911| 2911|
 238881| 5714485| -2148| 2148|
 238881| 5714486| -381| 381|
 238881| 5714487| -95| 95|
 238881| 5714488| -444| 444|
+----------+------------+-----------+-------------+
only showing top 20 rows

In [33]:
customer_credit = customer_credit.drop("DAYS_CREDIT")

In [34]:
customer_group = Window.partitionBy("SK_ID_CURR").orderBy("DAYS_CREDIT")
customer_credit = customer_credit.withColumn("PREV_VALUE", lag(customer_credit.DAYS_CREDIT_P).over(customer_group))

In [35]:
customer_credit.show()

+----------+------------+-----------+-------------+----------+
SK_ID_CURR|SK_ID_BUREAU|DAYS_CREDIT|DAYS_CREDIT_P|PREV_VALUE|
+----------+------------+-----------+-------------+----------+
 100053| 5723580| -2893| 2893| null|
 100053| 5723579| -2688| 2688| 2893|
 100053| 5723581| -2639| 2639| 2688|
 100053| 5723578| -2338| 2338| 2639|
 100053| 5723582| -1925| 1925| 2338|
 100053| 5723583| -1767| 1767| 1925|
 100053| 5723577| -1764| 1764| 1767|
 100568| 5719894| -1274| 1274| null|
 100568| 5719892| -556| 556| 1274|
 100568| 5719898| -544| 544| 556|
 100568| 5719897| -319| 319| 544|
 100568| 5719895| -279| 279| 319|
 100568| 5719896| -278| 278| 279|
 100568| 5719893| -207| 207| 278|
 100653| 5726283| -2474| 2474| null|
 100653| 5726284| -2258| 2258| 2474|
 100653| 5726285| -1677| 1677| 2258|
 100653| 5726287| -1564| 1564| 1677|
 100653| 5726282| -256| 256| 1564|
 100653| 5726288| -252| 252| 256|
+----------+------------+-----------+-------------+----------+
only showing top 20 rows

In [36]:
customer_credit = customer_credit.withColumn("DAYS_DIFF", when(isnull(customer_credit.PREV_VALUE - customer_credit.DAYS_CREDIT_P), 0)
                              .otherwise(customer_credit.PREV_VALUE - customer_credit.DAYS_CREDIT_P))

In [37]:
customer_credit.show()

+----------+------------+-------------+----------+---------+
SK_ID_CURR|SK_ID_BUREAU|DAYS_CREDIT_P|PREV_VALUE|DAYS_DIFF|
+----------+------------+-------------+----------+---------+
 100053| 5723580| 2893| null| 0|
 100053| 5723579| 2688| 2893| 205|
 100053| 5723581| 2639| 2688| 49|
 100053| 5723578| 2338| 2639| 301|
 100053| 5723582| 1925| 2338| 413|
 100053| 5723583| 1767| 1925| 158|
 100053| 5723577| 1764| 1767| 3|
 100568| 5719894| 1274| null| 0|
 100568| 5719892| 556| 1274| 718|
 100568| 5719898| 544| 556| 12|
 100568| 5719897| 319| 544| 225|
 100568| 5719895| 279| 319| 40|
 100568| 5719896| 278| 279| 1|
 100568| 5719893| 207| 278| 71|
 100653| 5726283| 2474| null| 0|
 100653| 5726284| 2258| 2474| 216|
 100653| 5726285| 1677| 2258| 581|
 100653| 5726287| 1564| 1677| 113|
 100653| 5726282| 256| 1564| 1308|
 100653| 5726288| 252| 256| 4|
+----------+------------+-------------+----------+---------+
only showing top 20 rows

In [38]:
bureau_10000 = bureau_10000.join(customer_credit,['SK_ID_CURR'], how ='left')

In [39]:
display(bureau_10000)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_BUREAU,DAYS_CREDIT_P,PREV_VALUE,DAYS_DIFF
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,5714468,43,203,160
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,5714465,203,203,0
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,5714464,203,208,5
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,5714463,208,273,65
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,5714467,273,497,224
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,5714462,497,629,132
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,5714466,629,null,0
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,5714468,43,203,160
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,5714465,203,203,0
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,5714464,203,208,5


d #### Feature 6 : % of loans per customer where end data for credit is past

Interpretting CREDIT_DAYS_ENDDATE

NEGATIVE VALUE - Credit date was in the past at time of application( Potential Red Flag !!! )

POSITIVE VALUE - Credit date is in the future at time of application ( Potential Good Sign !!!!)

In [41]:
def pos_neg_end_date_trans(x):
    if x < 0:
        y = 0
    else:
        y = 1    
    return y

In [42]:
pos_neg_end_date_trans_udf = udf( pos_neg_end_date_trans, StringType() )

In [43]:
bureau_10000 = bureau.limit(10000)
days_end_credit =  bureau_10000.select("SK_ID_CURR","DAYS_CREDIT_ENDDATE")

In [44]:
days_end_credit.show()

+----------+-------------------+
SK_ID_CURR|DAYS_CREDIT_ENDDATE|
+----------+-------------------+
 215354| -153.0|
 215354| 1075.0|
 215354| 528.0|
 215354| null|
 215354| 1197.0|
 215354| 27460.0|
 215354| 79.0|
 162297| -1684.0|
 162297| -811.0|
 162297| -484.0|
 162297| -180.0|
 162297| -629.0|
 162297| 5261.0|
 402440| 269.0|
 238881| -187.0|
 238881| -2607.0|
 238881| -1595.0|
 238881| null|
 238881| 1720.0|
 238881| -77.0|
+----------+-------------------+
only showing top 20 rows

In [45]:
days_end_credit = days_end_credit.withColumn( "CREDIT_ENDDATE_BINARY",
                   pos_neg_end_date_trans_udf( days_end_credit.DAYS_CREDIT_ENDDATE) )

In [46]:
days_end_credit.show()

+----------+-------------------+---------------------+
SK_ID_CURR|DAYS_CREDIT_ENDDATE|CREDIT_ENDDATE_BINARY|
+----------+-------------------+---------------------+
 215354| -153.0| 0|
 215354| 1075.0| 1|
 215354| 528.0| 1|
 215354| null| 0|
 215354| 1197.0| 1|
 215354| 27460.0| 1|
 215354| 79.0| 1|
 162297| -1684.0| 0|
 162297| -811.0| 0|
 162297| -484.0| 0|
 162297| -180.0| 0|
 162297| -629.0| 0|
 162297| 5261.0| 1|
 402440| 269.0| 1|
 238881| -187.0| 0|
 238881| -2607.0| 0|
 238881| -1595.0| 0|
 238881| null| 0|
 238881| 1720.0| 1|
 238881| -77.0| 0|
+----------+-------------------+---------------------+
only showing top 20 rows

In [47]:
days_end_credit_per = days_end_credit.groupBy("SK_ID_CURR").agg(avg("CREDIT_ENDDATE_BINARY").alias("CREDIT_ENDDATE_PERCENTAGE"))

In [48]:
days_end_credit_per.show()

+----------+-------------------------+
SK_ID_CURR|CREDIT_ENDDATE_PERCENTAGE|
+----------+-------------------------+
 341504| 0.5|
 197603| 0.2|
 330299| 0.07692307692307693|
 355377| 0.375|
 299495| 1.0|
 205426| 0.2222222222222222|
 197732| 0.0|
 176469| 0.5|
 341505| 0.125|
 349442| 0.6666666666666666|
 398701| 0.0|
 221642| 0.5|
 202808| 0.0|
 299700| 0.8888888888888888|
 238499| 0.5|
 255514| 0.4444444444444444|
 168529| 0.0|
 444842| 0.5|
 234818| 0.5|
 110203| 0.3333333333333333|
+----------+-------------------------+
only showing top 20 rows

In [49]:
bureau_10000 = bureau_10000.join(days_end_credit_per,['SK_ID_CURR'],how = 'left')

In [50]:
display(bureau_10000)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,CREDIT_ENDDATE_PERCENTAGE
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,0.7142857142857143
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,0.7142857142857143
215354,5714464,Active,currency 1,-203,0,528.0,null,null,0,464323.5,null,null,0.0,Consumer credit,-16,null,0.7142857142857143
215354,5714465,Active,currency 1,-203,0,null,null,null,0,90000.0,null,null,0.0,Credit card,-16,null,0.7142857142857143
215354,5714466,Active,currency 1,-629,0,1197.0,null,77674.5,0,2700000.0,null,null,0.0,Consumer credit,-21,null,0.7142857142857143
215354,5714467,Active,currency 1,-273,0,27460.0,null,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,null,0.7142857142857143
215354,5714468,Active,currency 1,-43,0,79.0,null,0.0,0,42103.8,42103.8,0.0,0.0,Consumer credit,-22,null,0.7142857142857143
162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.0,0.0,0.0,Consumer credit,-1710,null,0.16666666666666666
162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.7,0.0,0.0,0.0,Consumer credit,-840,null,0.16666666666666666
162297,5714471,Active,currency 1,-1146,0,-484.0,null,0.0,0,4500.0,0.0,0.0,0.0,Credit card,-690,null,0.16666666666666666


#### Feature 7 : Average number of days in which credit expires in future - indication of customer delinquency in future ??

We take only positive values of  ENDDATE since we are looking at Bureau Credit VALID IN FUTURE

In [53]:
days_end_credit_p = days_end_credit.where("CREDIT_ENDDATE_BINARY = 1")

In [54]:
days_end_credit_p.show()

+----------+-------------------+---------------------+
SK_ID_CURR|DAYS_CREDIT_ENDDATE|CREDIT_ENDDATE_BINARY|
+----------+-------------------+---------------------+
 215354| 1075.0| 1|
 215354| 528.0| 1|
 215354| 1197.0| 1|
 215354| 27460.0| 1|
 215354| 79.0| 1|
 162297| 5261.0| 1|
 402440| 269.0| 1|
 238881| 1720.0| 1|
 222183| 1008.0| 1|
 222183| 625.0| 1|
 222183| 1431.0| 1|
 222183| 1512.0| 1|
 136226| 551.0| 1|
 400486| 705.0| 1|
 435112| 1318.0| 1|
 435112| 680.0| 1|
 389599| 10285.0| 1|
 389599| 1253.0| 1|
 389599| 331.0| 1|
 389599| 277.0| 1|
+----------+-------------------+---------------------+
only showing top 20 rows

In [55]:
end_date_credit_window = Window.partitionBy("SK_ID_CURR").orderBy("DAYS_CREDIT_ENDDATE")
days_end_credit_p = days_end_credit_p.withColumn("PREV_VALUE", lag(days_end_credit_p.DAYS_CREDIT_ENDDATE).over(end_date_credit_window ))
days_end_credit_p = days_end_credit_p.withColumn("DAYS_ENDDATE_DIFF", when(isnull(days_end_credit_p.DAYS_CREDIT_ENDDATE -days_end_credit_p.PREV_VALUE), 0)
                              .otherwise(days_end_credit_p.DAYS_CREDIT_ENDDATE - days_end_credit_p.PREV_VALUE))

In [56]:
days_end_credit_p.show()

+----------+-------------------+---------------------+----------+-----------------+
SK_ID_CURR|DAYS_CREDIT_ENDDATE|CREDIT_ENDDATE_BINARY|PREV_VALUE|DAYS_ENDDATE_DIFF|
+----------+-------------------+---------------------+----------+-----------------+
 197603| 1233.0| 1| null| 0.0|
 330299| 268.0| 1| null| 0.0|
 341504| 82.0| 1| null| 0.0|
 355377| 49.0| 1| null| 0.0|
 355377| 128.0| 1| 49.0| 79.0|
 355377| 820.0| 1| 128.0| 692.0|
 176469| 361.0| 1| null| 0.0|
 205426| 71.0| 1| null| 0.0|
 205426| 911.0| 1| 71.0| 840.0|
 299495| 763.0| 1| null| 0.0|
 299495| 780.0| 1| 763.0| 17.0|
 299495| 855.0| 1| 780.0| 75.0|
 299495| 10178.0| 1| 855.0| 9323.0|
 341505| 1190.0| 1| null| 0.0|
 349442| 43.0| 1| null| 0.0|
 349442| 903.0| 1| 43.0| 860.0|
 110203| 149.0| 1| null| 0.0|
 186607| 137.0| 1| null| 0.0|
 186607| 1031.0| 1| 137.0| 894.0|
 221642| 1610.0| 1| null| 0.0|
+----------+-------------------+---------------------+----------+-----------------+
only showing top 20 rows

In [57]:
days_end_credit_p = days_end_credit_p.drop("DAYS_CREDIT_ENDDATE","CREDIT_ENDDATE_BINARY","PREV_VALUE")

In [58]:
bureau_10000 = bureau_10000.join(days_end_credit_p,['SK_ID_CURR'], how = 'left')

In [59]:
display(bureau_10000)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,CREDIT_ENDDATE_PERCENTAGE,DAYS_ENDDATE_DIFF
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,0.7142857142857143,26263.0
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,0.7142857142857143,122.0
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,0.7142857142857143,547.0
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,0.7142857142857143,449.0
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,0.7142857142857143,0.0
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,0.7142857142857143,26263.0
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,0.7142857142857143,122.0
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,0.7142857142857143,547.0
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,0.7142857142857143,449.0
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,0.7142857142857143,0.0


In [60]:
days_end_credit_p.show()

+----------+-----------------+
SK_ID_CURR|DAYS_ENDDATE_DIFF|
+----------+-----------------+
 197603| 0.0|
 330299| 0.0|
 341504| 0.0|
 355377| 0.0|
 355377| 79.0|
 355377| 692.0|
 176469| 0.0|
 205426| 0.0|
 205426| 840.0|
 299495| 0.0|
 299495| 17.0|
 299495| 75.0|
 299495| 9323.0|
 341505| 0.0|
 349442| 0.0|
 349442| 860.0|
 110203| 0.0|
 186607| 0.0|
 186607| 894.0|
 221642| 0.0|
+----------+-----------------+
only showing top 20 rows

In [61]:
avg_enddate_future_grp = days_end_credit_p.groupBy("SK_ID_CURR").agg(avg("DAYS_ENDDATE_DIFF").alias("AVG_ENDDATE_FUTURE"))

In [62]:
avg_enddate_future_grp.show()

+----------+------------------+
SK_ID_CURR|AVG_ENDDATE_FUTURE|
+----------+------------------+
 197603| 0.0|
 330299| 0.0|
 341504| 0.0|
 355377| 257.0|
 176469| 0.0|
 205426| 420.0|
 299495| 2353.75|
 341505| 0.0|
 349442| 430.0|
 110203| 0.0|
 186607| 447.0|
 221642| 0.0|
 234818| 208.5|
 238499| 0.0|
 252487| 0.0|
 253537| 108.0|
 255514| 55.25|
 299700| 1169.5|
 311139| 0.0|
 444842|238.33333333333334|
+----------+------------------+
only showing top 20 rows

In [63]:
bureau_10000 = bureau_10000.join(avg_enddate_future_grp,['SK_ID_CURR'], how = 'left')

In [64]:
display(bureau_10000)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,CREDIT_ENDDATE_PERCENTAGE,DAYS_ENDDATE_DIFF,AVG_ENDDATE_FUTURE
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,0.7142857142857143,26263.0,5476.2
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,0.7142857142857143,122.0,5476.2
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,0.7142857142857143,547.0,5476.2
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,0.7142857142857143,449.0,5476.2
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,0.7142857142857143,0.0,5476.2
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,0.7142857142857143,26263.0,5476.2
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,0.7142857142857143,122.0,5476.2
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,0.7142857142857143,547.0,5476.2
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,0.7142857142857143,449.0,5476.2
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,0.7142857142857143,0.0,5476.2


d #### Feature 8 : Debt over credit ratio
The Ratio of Total Debt to Total Credit for each Customer

A High value may be a red flag indicative of potential default

In [66]:
bureau_10000 = bureau.limit(10000)

In [67]:
 AMT_CREDIT_SUM_DEBT = bureau_10000.filter(bureau_10000.AMT_CREDIT_SUM_DEBT.isNull())

In [68]:
AMT_CREDIT_SUM_DEBT.count()

Out[ 42 ]: 1540

In [69]:
AMT_CREDIT_SUM = bureau_10000.filter(bureau_10000.AMT_CREDIT_SUM.isNull())

In [70]:
AMT_CREDIT_SUM.count()

Out[ 45 ]: 0

In [71]:
bureau_10000 = bureau_10000.na.fill({'AMT_CREDIT_SUM_DEBT': 0, 'AMT_CREDIT_SUM': 0})

In [72]:
 AMT_CREDIT_SUM_DEBT = bureau_10000.filter(bureau_10000.AMT_CREDIT_SUM_DEBT.isNull())

In [73]:
AMT_CREDIT_SUM_DEBT.count()

Out[ 49 ]: 0

In [74]:
grp1 = bureau_10000.select('SK_ID_CURR', 'AMT_CREDIT_SUM').groupBy("SK_ID_CURR").agg(sum("AMT_CREDIT_SUM").alias("TOTAL_CUSTOMER_DEBT"))

In [75]:
grp1.show()

+----------+-------------------+
SK_ID_CURR|TOTAL_CUSTOMER_DEBT|
+----------+-------------------+
 215354| 3792750.3|
 162297| 8230386.15|
 402440| 89910.0|
 238881| 1060239.06|
 222183| 5880550.5|
 426155| 328555.44|
 136226| 222750.0|
 400486| 702517.5|
 435112| 2160554.625|
 452585| 117261.0|
 389599| 1781320.5|
 242993| 1402816.5|
 303740| 568911.06|
 234931| 1566247.5|
 311918| 91984.5|
 119939| 6697561.5|
 388421| 2082586.5|
 419892| 3099118.5|
 387080| 1686354.75|
 293201| 649591.9199999999|
+----------+-------------------+
only showing top 20 rows

In [76]:
grp2 = bureau_10000.groupBy("SK_ID_CURR").agg(sum("AMT_CREDIT_SUM_DEBT").alias("TOTAL_CUSTOMER_CREDIT"))

In [77]:
grp2.show()

+----------+---------------------+
SK_ID_CURR|TOTAL_CUSTOMER_CREDIT|
+----------+---------------------+
 215354| 284463.18|
 162297| 0.0|
 402440| 76905.0|
 238881| 552730.5|
 222183| 1185081.885|
 426155| 0.0|
 136226| 0.0|
 400486| 108315.0|
 435112| 1042296.57|
 452585| 54000.0|
 389599| 795703.5|
 242993| 59809.5|
 303740| 263056.5|
 234931| 17982.0|
 311918| 20835.0|
 119939| 495519.12|
 388421| 662139.0|
 419892| 2739622.5|
 387080| 613084.5|
 293201| -701.28|
+----------+---------------------+
only showing top 20 rows

In [78]:
bureau_10000 = bureau_10000.join(grp1,['SK_ID_CURR'],'left')

In [79]:
display(bureau_10000)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,TOTAL_CUSTOMER_DEBT
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,3792750.3
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,3792750.3
215354,5714464,Active,currency 1,-203,0,528.0,null,null,0,464323.5,0.0,null,0.0,Consumer credit,-16,null,3792750.3
215354,5714465,Active,currency 1,-203,0,null,null,null,0,90000.0,0.0,null,0.0,Credit card,-16,null,3792750.3
215354,5714466,Active,currency 1,-629,0,1197.0,null,77674.5,0,2700000.0,0.0,null,0.0,Consumer credit,-21,null,3792750.3
215354,5714467,Active,currency 1,-273,0,27460.0,null,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,null,3792750.3
215354,5714468,Active,currency 1,-43,0,79.0,null,0.0,0,42103.8,42103.8,0.0,0.0,Consumer credit,-22,null,3792750.3
162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.0,0.0,0.0,Consumer credit,-1710,null,8230386.15
162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.7,0.0,0.0,0.0,Consumer credit,-840,null,8230386.15
162297,5714471,Active,currency 1,-1146,0,-484.0,null,0.0,0,4500.0,0.0,0.0,0.0,Credit card,-690,null,8230386.15


In [80]:
bureau_10000 = bureau_10000.join(grp2,['SK_ID_CURR'],'left')

In [81]:
bureau_10000 = bureau_10000.withColumn("DEBT_CREDIT_RATIO",col("TOTAL_CUSTOMER_DEBT")/col("TOTAL_CUSTOMER_CREDIT"))

In [82]:
bureau_10000 = bureau_10000.drop("TOTAL_CUSTOMER_DEBT", "TOTAL_CUSTOMER_CREDIT")

In [83]:
display(bureau_10000)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null
215354,5714464,Active,currency 1,-203,0,528.0,null,null,0,464323.5,0.0,null,0.0,Consumer credit,-16,null
215354,5714465,Active,currency 1,-203,0,null,null,null,0,90000.0,0.0,null,0.0,Credit card,-16,null
215354,5714466,Active,currency 1,-629,0,1197.0,null,77674.5,0,2700000.0,0.0,null,0.0,Consumer credit,-21,null
215354,5714467,Active,currency 1,-273,0,27460.0,null,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,null
215354,5714468,Active,currency 1,-43,0,79.0,null,0.0,0,42103.8,42103.8,0.0,0.0,Consumer credit,-22,null
162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.0,0.0,0.0,Consumer credit,-1710,null
162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.7,0.0,0.0,0.0,Consumer credit,-840,null
162297,5714471,Active,currency 1,-1146,0,-484.0,null,0.0,0,4500.0,0.0,0.0,0.0,Credit card,-690,null


#### Feature 9 : Overdue over debt ration
What fraction of total Debt is overdue per customer?

A high value could indicate a potential DEFAULT

In [85]:
bureau_10000 = bureau.limit(10000)
bureau_10000 = bureau_10000.na.fill({'AMT_CREDIT_SUM_DEBT': 0, 'AMT_CREDIT_SUM_OVERDUE': 0})

In [86]:
grp1 = bureau_10000.select('SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT').groupBy("SK_ID_CURR").agg(sum("AMT_CREDIT_SUM_DEBT").alias("TOTAL_CUSTOMER_DEBT"))
grp2 = bureau_10000.groupBy("SK_ID_CURR").agg(sum("AMT_CREDIT_SUM_OVERDUE").alias("TOTAL_CUSTOMER_OVERDUE"))

In [87]:
grp1.show()

+----------+-------------------+
SK_ID_CURR|TOTAL_CUSTOMER_DEBT|
+----------+-------------------+
 215354| 284463.18|
 162297| 0.0|
 402440| 76905.0|
 238881| 552730.5|
 222183| 1185081.885|
 426155| 0.0|
 136226| 0.0|
 400486| 108315.0|
 435112| 1042296.57|
 452585| 54000.0|
 389599| 795703.5|
 242993| 59809.5|
 303740| 263056.5|
 234931| 17982.0|
 311918| 20835.0|
 119939| 495519.12|
 388421| 662139.0|
 419892| 2739622.5|
 387080| 613084.5|
 293201| -701.28|
+----------+-------------------+
only showing top 20 rows

In [88]:
grp2.show(100)

+----------+----------------------+
SK_ID_CURR|TOTAL_CUSTOMER_OVERDUE|
+----------+----------------------+
 215354| 0.0|
 162297| 0.0|
 402440| 0.0|
 238881| 0.0|
 222183| 0.0|
 426155| 0.0|
 136226| 0.0|
 400486| 0.0|
 435112| 0.0|
 452585| 0.0|
 389599| 0.0|
 242993| 0.0|
 303740| 0.0|
 234931| 0.0|
 311918| 0.0|
 119939| 0.0|
 388421| 0.0|
 419892| 0.0|
 387080| 0.0|
 293201| 0.0|
 338464| 0.0|
 444864| 0.0|
 253882| 0.0|
 335122| 0.0|
 351919| 0.0|
 406724| 0.0|
 389074| 0.0|
 373324| 0.0|
 286014| 0.0|
 241216| 0.0|
 166034| 0.0|
 399459| 0.0|
 173404| 0.0|
 280788| 0.0|
 312983| 0.0|
 272136| 0.0|
 415899| 0.0|
 451241| 0.0|
 383257| 0.0|
 435368| 0.0|
 227903| 0.0|
 228777| 0.0|
 304102| 0.0|
 266501| 0.0|
 417300| 0.0|
 416951| 0.0|
 225481| 0.0|
 148089| 0.0|
 390784| 0.0|
 346919| 0.0|
 233234| 0.0|
 181326| 0.0|
 297876| 0.0|
 278778| 0.0|
 198319| 0.0|
 333498| 0.0|
 152965| 0.0|
 346232| 0.0|
 348530| 0.0|
 271160| 0.0|
 182982| 0.0|
 259670| 0.0|
 354574| 0.0|
 450549| 0.0|
 251360| 0.0|
 262765| 0.0|
 452711| 0.0|
 374159| 0.0|
 456062| 0.0|
 299495| 0.0|
 282297| 231.525|
 331446| 0.0|
 159650| 0.0|
 273814| 0.0|
 192645| 0.0|
 132745| 0.0|
 381884| 0.0|
 228839| 0.0|
 221368| 0.0|
 256578| 0.0|
 414912| 0.0|
 391685| 0.0|
 317276| 0.0|
 115001| 0.0|
 233056| 0.0|
 346739| 0.0|
 221642| 0.0|
 205426| 0.0|
 432545| 0.0|
 118247| 0.0|
 166350| 0.0|
 388014| 0.0|
 189122| 0.0|
 166358| 0.0|
 157200| 0.0|
 142568| 0.0|
 356727| 0.0|
 250551| 0.0|
 224915| 0.0|
 323674| 0.0|
+----------+----------------------+
only showing top 100 rows

In [89]:
bureau_10000 = bureau_10000.join(grp1,['SK_ID_CURR'],how = 'left')

In [90]:
bureau_10000 = bureau_10000.join(grp2,['SK_ID_CURR'],how = 'left')

In [91]:
bureau_10000 = bureau_10000.withColumn("OVERDUE_DEBT_RATIO",round(col("TOTAL_CUSTOMER_OVERDUE")/col("TOTAL_CUSTOMER_DEBT"),2))

In [92]:
bureau_10000.select("OVERDUE_DEBT_RATIO").show()

+------------------+
OVERDUE_DEBT_RATIO|
+------------------+
 0.0|
 0.0|
 0.0|
 0.0|
 0.0|
 0.0|
 0.0|
 null|
 null|
 null|
 null|
 null|
 null|
 0.0|
 0.0|
 0.0|
 0.0|
 0.0|
 0.0|
 0.0|
+------------------+
only showing top 20 rows

In [93]:
display(bureau_10000)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,TOTAL_CUSTOMER_DEBT,TOTAL_CUSTOMER_OVERDUE,OVERDUE_DEBT_RATIO
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,284463.18,0.0,0.0
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,284463.18,0.0,0.0
215354,5714464,Active,currency 1,-203,0,528.0,null,null,0,464323.5,0.0,null,0.0,Consumer credit,-16,null,284463.18,0.0,0.0
215354,5714465,Active,currency 1,-203,0,null,null,null,0,90000.0,0.0,null,0.0,Credit card,-16,null,284463.18,0.0,0.0
215354,5714466,Active,currency 1,-629,0,1197.0,null,77674.5,0,2700000.0,0.0,null,0.0,Consumer credit,-21,null,284463.18,0.0,0.0
215354,5714467,Active,currency 1,-273,0,27460.0,null,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,null,284463.18,0.0,0.0
215354,5714468,Active,currency 1,-43,0,79.0,null,0.0,0,42103.8,42103.8,0.0,0.0,Consumer credit,-22,null,284463.18,0.0,0.0
162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.0,0.0,0.0,Consumer credit,-1710,null,0.0,0.0,null
162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.7,0.0,0.0,0.0,Consumer credit,-840,null,0.0,0.0,null
162297,5714471,Active,currency 1,-1146,0,-484.0,null,0.0,0,4500.0,0.0,0.0,0.0,Credit card,-690,null,0.0,0.0,null


#### Feature 10 : Average number of loans prolonged

In [95]:
bureau_10000 = bureau.limit(10000)
bureau_10000 = bureau_10000.na.fill({'CNT_CREDIT_PROLONG': 0})
grp = bureau_10000.select('SK_ID_CURR', 'CNT_CREDIT_PROLONG').groupBy("SK_ID_CURR").agg(avg("CNT_CREDIT_PROLONG").alias("AVG_CREDITDAYS_PROLONGED"))

In [96]:
grp.show()

+----------+------------------------+
SK_ID_CURR|AVG_CREDITDAYS_PROLONGED|
+----------+------------------------+
 215354| 0.0|
 162297| 0.0|
 402440| 0.0|
 238881| 0.0|
 222183| 0.0|
 426155| 0.0|
 136226| 0.0|
 400486| 0.0|
 435112| 0.0|
 452585| 0.0|
 389599| 0.0|
 242993| 0.0|
 303740| 0.0|
 234931| 0.0|
 311918| 0.0|
 119939| 0.0|
 388421| 0.0|
 419892| 0.0|
 387080| 0.0|
 293201| 0.0|
+----------+------------------------+
only showing top 20 rows

In [97]:
bureau_10000 = bureau_10000.join(grp,['SK_ID_CURR'],'left')

In [98]:
display(bureau_10000)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,AVG_CREDITDAYS_PROLONGED
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,null,0,91323.0,0.0,null,0.0,Consumer credit,-131,null,0.0
215354,5714463,Active,currency 1,-208,0,1075.0,null,null,0,225000.0,171342.0,null,0.0,Credit card,-20,null,0.0
215354,5714464,Active,currency 1,-203,0,528.0,null,null,0,464323.5,null,null,0.0,Consumer credit,-16,null,0.0
215354,5714465,Active,currency 1,-203,0,null,null,null,0,90000.0,null,null,0.0,Credit card,-16,null,0.0
215354,5714466,Active,currency 1,-629,0,1197.0,null,77674.5,0,2700000.0,null,null,0.0,Consumer credit,-21,null,0.0
215354,5714467,Active,currency 1,-273,0,27460.0,null,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,null,0.0
215354,5714468,Active,currency 1,-43,0,79.0,null,0.0,0,42103.8,42103.8,0.0,0.0,Consumer credit,-22,null,0.0
162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.0,0.0,0.0,Consumer credit,-1710,null,0.0
162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.7,0.0,0.0,0.0,Consumer credit,-840,null,0.0
162297,5714471,Active,currency 1,-1146,0,-484.0,null,0.0,0,4500.0,0.0,0.0,0.0,Credit card,-690,null,0.0
